<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ALBERT_Trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical  
import string
import requests

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
import tokenization

     |████████████████████████████████| 1.2MB 11.9MB/s 


In [ ]:
def albert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = pooled_output
    out = clf_output
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/albert_en_base/1"
albert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 3.68 s, sys: 560 ms, total: 4.24 s
Wall time: 12.1 s


In [ ]:
with open('/content/arranged_processed_data.pkl', 'rb') as f:
  processed_data = pickle.load(f)
# with open('/content/arranged_processed_label.pkl', 'rb') as f:
#   processed_label = pickle.load(f)
# with open('/content/vader_encodings.pkl', 'rb') as f:
#   vader_encodings = pickle.load(f)

In [ ]:
processed_data=np.asarray(processed_data)
Y=to_categorical(processed_label, num_classes=6)

In [ ]:
sp_model_file = albert_layer.resolved_object.sp_model_file.asset_path.numpy()
tokenizer = tokenization.FullSentencePieceTokenizer(sp_model_file)

In [ ]:
train_input = albert_encode(processed_data, tokenizer, max_len=160)
train_labels = Y

In [ ]:
model = build_model(albert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 11683584    input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
y_pred=model.predict(train_input, batch_size=1)

In [ ]:
print(type(y_pred))

<class 'numpy.ndarray'>


In [ ]:
print(type(vader_encodings))

<class 'list'>


In [ ]:
albert_encodings = list(y_pred)

In [ ]:
print(len(albert_encodings[0]))

768


In [ ]:
print(vader_encodings[0])

[0.0, 0.625, 0.375, 0.8126]


In [ ]:
print(list(albert_encodings[0]).extend(vader_encodings[0]))

None


In [ ]:
train_input=[]
for i in range(0,len(vader_encodings)):
  temp=[]
  for j in albert_encodings[i]:
    temp.append(j)
  for j in vader_encodings[i]:
    temp.append(j)
  train_input.append(np.asarray(temp))

In [ ]:
train_input=np.asarray(train_input)

In [ ]:
model = Sequential()
model.add(Dense(100, input_dim=772, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_input, train_labels, epochs=100, batch_size=1)

In [2]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

     |████████████████████████████████| 133kB 7.5MB/s 


In [ ]:
def addFeatures(sentences):
  vader_features=[]
  for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    temp=[]
    temp.append(sentence)
    if vs['neg'] > 0.2:
      temp.append('negative')
    if vs['pos'] > 0.2:
      temp.append('positive')
    if vs['neu'] > 0.2:
      temp.append('neutral')
    if vs['compound'] > 0.2:
      temp.append('compound')
    temp=' '.join(temp)
    vader_features.append(temp)
  return vader_features

In [ ]:
vader_features = addFeatures(processed_data)

In [ ]:
print(vader_features[0])

thank you for calling sprint . we care about everybody . how can i help you ? positive neutral compound


In [ ]:
import pickle
with open('vader_featured_sentences.pkl', 'wb') as f:
  pickle.dump(vader_features, f)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [55]:
import pickle
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)
with open('/content/gdrive/MyDrive/iemocap/train/conversation_length.pkl', 'rb') as f:
    train_convlen = pickle.load(f)
with open('/content/gdrive/MyDrive/iemocap/test/conversation_length.pkl', 'rb') as f:
    test_convlen = pickle.load(f)

In [ ]:
for i in range(0,len(data)):
  print(len(data[i]))
  print("It is sentence : " + str(i))

In [ ]:
for i in range(0,len(test_data)):
  print(len(test_data[i]))
  print("It is sentence : " + str(i))

In [56]:
test_data[20]=data[81]
test_labels[20]=labels[81]
test_data[21]=data[94]
test_labels[21]=labels[94]
test_data[22]=data[94]
test_labels[22]=labels[94]
test_data[23]=data[84]
test_labels[23]=labels[84]
test_data[24]=data[62]
test_labels[24]=labels[62]
test_data[25]=data[80]
test_labels[25]=labels[80]
test_data[26]=data[38]
test_labels[26]=labels[38]
test_data[27]=data[93]
test_labels[27]=labels[93]
test_data[16]=data[90]
test_labels[16]=labels[90]
test_data[17]=data[53]
test_labels[17]=labels[53]
test_data[18]=data[29]
test_labels[18]=labels[29]
test_data[19]=data[46]
test_labels[19]=labels[46]
test_data[7]=data[75]
test_labels[7]=labels[75]

In [33]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [57]:
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [58]:
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [59]:
test_vader_features = test_processed_data

In [60]:
with open('new_test_vader_features.pkl', 'wb') as f:
  pickle.dump(test_vader_features, f)
with open('new_test_vader_labels.pkl', 'wb') as f:
  pickle.dump(test_processed_label, f)